In [8]:
# kimimaro kernel
import numpy as np
from funlib.geometry import Roi
from funlib.persistence import prepare_ds

labels = np.load("/nrs/cellmap/ackermand/tests/connectomics.npy")
ds = prepare_ds(
    "/nrs/cellmap/ackermand/tests/skeletonization.n5",
    "connectomics",
    total_roi=Roi((0, 0, 0), labels.shape),
    voxel_size=(1, 1, 1),
    write_size=(128, 128, 128),
    dtype=labels.dtype,
)
ds.data[:] = labels

In [ ]:
np.unique()

In [22]:
from funlib.persistence import open_ds

con = open_ds("/nrs/cellmap/ackermand/tests/skeletonization.n5", "connectomics")
skel = open_ds(
    "/nrs/cellmap/ackermand/tests/skeletonization.n5", "connectomics_skeleton"
)
set(np.unique(con.data)) - set(np.unique(skel.data))

set()

In [4]:
import kimimaro

skels = kimimaro.skeletonize(
    labels,
    teasar_params={
        "scale": 1.5,
        "const": 300,  # physical units
        "pdrf_scale": 100000,
        "pdrf_exponent": 4,
        "soma_acceptance_threshold": 3500,  # physical units
        "soma_detection_threshold": 750,  # physical units
        "soma_invalidation_const": 300,  # physical units
        "soma_invalidation_scale": 2,
        "max_paths": 300,  # default None
    },
    # object_ids=[ ... ], # process only the specified labels
    # extra_targets_before=[ (27,33,100), (44,45,46) ], # target points in voxels
    # extra_targets_after=[ (27,33,100), (44,45,46) ], # target points in voxels
    dust_threshold=1000,  # skip connected components with fewer than this many voxels
    anisotropy=(16, 16, 40),  # default True
    fix_branching=True,  # default True
    fix_borders=True,  # default True
    fill_holes=False,  # default False
    fix_avocados=False,  # default False
    progress=True,  # default False, show progress bar
    parallel=1,  # <= 0 all cpu, 1 single process, 2+ multiprocess
    parallel_chunk_size=100,  # how many skeletons to process before updating progress bar
)

Skeletonizing Labels: 100%|██████████| 2124/2124 [08:58<00:00,  3.94it/s]  


In [3]:
# kimimaro kernel
import numpy as np
from funlib.geometry import Roi, Coordinate
from funlib.persistence import open_ds

ds = open_ds("/nrs/cellmap/ackermand/cellmap/crop_jrc_mus-liver-zon-1.n5", "mito")
test_array = ds.to_ndarray(
    Roi(
        (61952 + 212 * 3 * 8, 35072 + 212 * 3 * 8, 120000 + 212 * 3 * 8)[::-1],
        Coordinate(8 * 512, 8 * 512, 8 * 512),
    )
)

In [30]:
from importlib import reload
import kimimaro

reload(kimimaro)
skels = kimimaro.skeletonize(
    test_array,
    teasar_params={
        "scale": 1.5,
        "const": 300,  # physical units
        "pdrf_scale": 100000,
        "pdrf_exponent": 4,
        "soma_acceptance_threshold": 3500,  # physical units
        "soma_detection_threshold": 750,  # physical units
        "soma_invalidation_const": 300,  # physical units
        "soma_invalidation_scale": 2,
        "max_paths": 300,  # default None
    },
    # object_ids=[ ... ], # process only the specified labels
    # extra_targets_before=[ (27,33,100), (44,45,46) ], # target points in voxels
    # extra_targets_after=[ (27,33,100), (44,45,46) ], # target points in voxels
    dust_threshold=0,  # skip connected components with fewer than this many voxels
    anisotropy=(8, 8, 8),  # default True
    fix_branching=True,  # default True
    fix_borders=True,  # default True
    fill_holes=False,  # default False
    fix_avocados=False,  # default False
    progress=True,  # default False, show progress bar
    parallel=2,  # <= 0 all cpu, 1 single process, 2+ multiprocess
    parallel_chunk_size=100,  # how many skeletons to process before updating progress bar
)

2024-03-27 13:57:39.361 INFO     start
2024-03-27 13:57:39.450 INFO     0
2024-03-27 13:57:39.451 INFO     1
2024-03-27 13:57:39.535 INFO     2
2024-03-27 13:57:39.645 INFO     3
2024-03-27 13:57:39.701 INFO     4
Skeletonizing Labels: 100%|██████████| 1/1 [00:00<00:00, 29.03it/s]


In [13]:
# kimimaro kernel
import numpy as np
from funlib.geometry import Roi, Coordinate
from funlib.persistence import open_ds
from ackermand.skeletonization.dask_skeletonization import Skeletonize

ds = open_ds("/nrs/cellmap/ackermand/cellmap/crop_jrc_mus-liver-zon-1.n5", "mito")
sk = Skeletonize(
    ds,
    total_roi=Roi(
        (61952, 35072, 120000)[::-1],
        Coordinate(8 * 6000, 8 * 6000, 8 * 6000),
    ),
    read_write_roi=Roi((0, 0, 0), Coordinate(8 * 512, 8 * 512, 8 * 512)),
)  # , Roi((61952+212*3*8, 35072+212*3*8, 120000+212*3*8)[::-1], Coordinate(8 * 212, 8 * 212, 8 * 212)) )
sk.get_skeletons()

block_processing ▶:   0%|          | 0/1728 [00:00<?, ?blocks/s]

In [2]:
import navis
import skeletor as sk


def simplify_skeleton(kimimaro_skel):
    # to simplify skeletons: https://github.com/navis-org/skeletor/issues/38#issuecomment-1491714639
    n = navis.TreeNeuron(kimimaro_skel.to_swc(), soma=None)
    ds = navis.downsample_neuron(n, downsampling_factor=20)

    # to renumber nodes
    # https://github.com/navis-org/skeletor/issues/26#issuecomment-1086965768
    swc = navis.io.swc_io.make_swc_table(ds)
    # We also need to rename some columns
    swc = swc.rename(
        {
            "PointNo": "node_id",
            "Parent": "parent_id",
            "X": "x",
            "Y": "y",
            "Z": "z",
            "Radius": "radius",
        },
        axis=1,
    ).drop("Label", axis=1)
    # Skeletor excepts node IDs to start with 0, but navis starts at 1 for SWC
    swc["node_id"] -= 1
    swc.loc[swc.parent_id > 0, "parent_id"] -= 1
    # Create the skeletor.Skeleton
    skeletor_skel = sk.Skeleton(swc)
    return skeletor_skel

In [3]:
import os
import pickle
from cloudvolume import Skeleton
import kimimaro
from tqdm import tqdm

from dask.distributed import Client, progress

client = Client(threads_per_worker=1, n_workers=10)

tmpdirname = "/nrs/cellmap/ackermand/tests/tmp/"
all_skels = {}
count = 0
for skeleton_id in tqdm(os.listdir(tmpdirname)):
    block_skeletons = []
    for skeleton_file in os.listdir(f"{tmpdirname}/{skeleton_id}"):
        with open(f"{tmpdirname}/{skeleton_id}/{skeleton_file}", "rb") as f:
            block_skeleton = pickle.load(f)
            block_skeletons.append(block_skeleton)
            if skeleton_id == "14654803557897":
                print(len(block_skeleton.vertices))
                count += len(block_skeleton.vertices)
        skel = Skeleton.simple_merge(block_skeletons)
        skel = kimimaro.postprocess(
            skel,
            dust_threshold=0,
            tick_threshold=80,  # physical units  # physical units
        )
        all_skels[int(skeleton_id)] = simplify_skeleton(skel)

  0%|          | 2/28922 [00:00<30:23, 15.86it/s]

  3%|▎         | 750/28922 [00:55<1:04:39,  7.26it/s]

176
56
361
770
180
495
28
850
144
37
182
205


 16%|█▋        | 4703/28922 [05:59<30:53, 13.07it/s]  


KeyboardInterrupt: 

In [47]:
all_skels[14654803557897] = skel2

In [51]:
from dataclasses import dataclass
from neuroglancer import coordinate_space
import json
import os
import numpy as np

metadata = {
    "@type": "neuroglancer_skeletons",
    "transform": [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
    "segment_properties": "segment_properties",
}

os.makedirs("skeletons/segment_properties", exist_ok=True)
with open(os.path.join("skeletons/", "info"), "w") as f:
    f.write(json.dumps(metadata))


@dataclass
class Source:
    vertex_attributes = []


from neuroglancer.skeleton import Skeleton

for skel_id, skel in all_skels.items():
    with open(f"skeletons/{skel_id}", "wb") as f:
        # need to flip to get in xyz
        skel = Skeleton(np.fliplr(skel.vertices), skel.edges)
        encoded = skel.encode(Source())
        f.write(encoded)

segment_properties = {
    "@type": "neuroglancer_segment_properties",
    "inline": {
        "ids": [str(skel_id) for skel_id in list(all_skels.keys())],
        "properties": [
            {"id": "label", "type": "label", "values": [""] * len(all_skels)}
        ],
    },
}
with open(f"skeletons/segment_properties/info", "w") as f:
    f.write(json.dumps(segment_properties))

In [49]:
all_skels[14654803557897]

<Skeleton(vertices=(327, 3), edges=(326, 2), method=None)>

In [21]:
skel_id, skel = list(all_skels.items())[0]
skel.id

11252425167364

In [16]:
from neuroglancer import coordinate_space

cs = coordinate_space.CoordinateSpace
cs.rank
cs.to_json()

TypeError: CoordinateSpace.to_json() missing 1 required positional argument: 'self'

In [7]:
import neuroglancer
import numpy as np
from funlib.geometry import Roi, Coordinate
from funlib.persistence import open_ds


class SkeletonSource(neuroglancer.skeleton.SkeletonSource):
    def __init__(self, dimensions):
        super().__init__(dimensions)
        self.vertex_attributes["affinity"] = neuroglancer.skeleton.VertexAttributeInfo(
            data_type=np.float32,
            num_components=1,
        )
        self.vertex_attributes["affinity2"] = neuroglancer.skeleton.VertexAttributeInfo(
            data_type=np.float32,
            num_components=1,
        )

    def get_skeleton(self, i):
        skel = all_skels[i]
        vertex_positions = skel.vertices
        edges = skel.edges
        gen = np.random.default_rng()
        return neuroglancer.skeleton.Skeleton(
            vertex_positions=vertex_positions,
            edges=edges,
            vertex_attributes=dict(
                affinity=gen.random(len(vertex_positions)),
                affinity2=gen.random(len(vertex_positions)),
                affinity3=gen.random(len(vertex_positions)),
            ),
        )


neuroglancer.set_server_bind_address("0.0.0.0")
viewer = neuroglancer.Viewer()
dimensions = neuroglancer.CoordinateSpace(
    names=["z", "y", "x"],
    units="nm",
    scales=[1, 1, 1],
)
with viewer.txn() as s:
    s.layers.append(
        name="a",
        layer=neuroglancer.SegmentationLayer(
            source=[
                neuroglancer.LocalVolume(
                    data=open_ds(
                        "/nrs/cellmap/ackermand/cellmap/crop_jrc_mus-liver-zon-1.n5",
                        "mito",
                    ),
                    dimensions=dimensions,
                ),
                SkeletonSource(dimensions),
            ],
            skeleton_shader="void main() { emitRGB(colormapJet(affinity)); }",
            selected_alpha=0,
            not_selected_alpha=0,
            segments=list(all_skels.keys()),
        ),
    )
    # s.layers.append(
    #     name="mesh",
    #     layer=neuroglancer.SegmentationLayer(
    #         source="precomputed://https://cellmap-vm1.int.janelia.org/nrs/ackermand/meshes/multiresolution/crop_jrc_mus-liver-zon-1/mito/multires"
    #     ),
    #     segments=list(all_skels.keys()),
    # )

    # Can adjust the skeleton rendering options
    s.layers[0].skeleton_rendering.mode2d = "lines"
    s.layers[0].skeleton_rendering.line_width2d = 3
    # s.layers[0].skeleton_rendering.mode3d = "lines_and_points"
    # s.layers[0].skeleton_rendering.line_width3d = 10

    # Can adjust visibility of layer side panel
    s.selected_layer.layer = "a"
    s.selected_layer.visible = True

# ap = argparse.ArgumentParser()
# neuroglancer.cli.add_server_arguments(ap)
# args = ap.parse_args()
# neuroglancer.cli.handle_server_arguments(args)
print(viewer)

http://ackermand-ws2.hhmi.org:33847/v/aa5b546f1c392f0d341044d750533df8d426e6d8/


ERROR:tornado.application:Uncaught exception GET /neuroglancer/mesh/aa5b546f1c392f0d341044d750533df8d426e6d8.6da39f6b30d9afef7c4fd7b66ace4c3c2361d020/12603007752198 (10.60.0.240)
HTTPServerRequest(protocol='http', host='ackermand-ws2.hhmi.org:33847', method='GET', uri='/neuroglancer/mesh/aa5b546f1c392f0d341044d750533df8d426e6d8.6da39f6b30d9afef7c4fd7b66ace4c3c2361d020/12603007752198', version='HTTP/1.1', remote_ip='10.60.0.240')
Traceback (most recent call last):
  File "/groups/scicompsoft/home/ackermand/miniconda3/envs/kimimaro/lib/python3.10/site-packages/tornado/web.py", line 1704, in _execute
    result = await result
  File "/groups/scicompsoft/home/ackermand/miniconda3/envs/kimimaro/lib/python3.10/site-packages/neuroglancer/server.py", line 453, in get
    encoded_mesh = await asyncio.wrap_future(
  File "/groups/scicompsoft/home/ackermand/miniconda3/envs/kimimaro/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File

In [15]:
from neuroglancer.skeleton import Skeleton

Skeleton()

88

In [ ]:
import daisy
from funlib.persistence import Array, open_ds
from funlib.geometry import Roi, Coordinate
import numpy as np
import tempfile
import pickle
import os
import json
import logging
import kimimaro

from cloudvolume import Skeleton

Skeleton.simple_merge()

In [ ]:
import dask.diagnostics
import dask.array as da
from dask.distributed import Client, LocalCluster
import numpy as np
from scipy.ndimage import binary_erosion
import socket
import itertools
from funlib.persistence import open_ds
from funlib.geometry import Roi
import itertools
from fibsem_tools import read_xarray, read, read_dask

# target array to be processed block-by-block
# use `da.overlap.overlap` for overlapped blocks
with LocalCluster(n_workers=8, threads_per_worker=1) as cluster:
    with Client(cluster) as client:
        print(
            f'Click here to monitor job: {client.dashboard_link.replace("127.0.0.1", socket.gethostname())}'
        )

        roi = Roi((19952, 9736, 153344), (13464, 14064, 15104)).snap_to_grid((8, 8, 8))
        start_original = np.array(roi.begin / 8).astype(int)
        end_original = np.array(roi.end / 8).astype(int)

        # need to snap to 256 since we are using that for chunk size and because chunks start at 0,0,0
        # in voxels
        roi = roi.snap_to_grid((256 * 8, 256 * 8, 256 * 8))

        segmentation_xarray = read_xarray(
            "/nrs/cellmap/ackermand/cellmap/crop_jrc_mus-liver-zon-1.n5/mito",
            chunks=(512, 512, 512),
        )

        # function that returns a python dictionary
        def get_overlap_dict(true_components, test_components, prediction_mask):
            print(block)

    # list comprehension to iterate over the blocks, lazily
    delayed_result = [
        dask.delayed(get_overlap_dict)(
            gt_da.blocks[inds],
            test_da.blocks[inds],
            prediction_mask_da.blocks[
                inds
            ],  # tuple([int(ind // 16) for ind in inds])],
        )
        for inds in itertools.product(*map(range, segmentation_xarray.chunks.shape))
    ]

    result = dask.delayed(list)(delayed_result)
    with dask.diagnostics.ProgressBar():
        res = result.compute()

import pickle

with open("validation_processing_result.pickle", "wb") as handle:
    pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [1]:
import dask.array as da

segmentations_da = da.from_zarr(
    "/nrs/cellmap/ackermand/cellmap/crop_jrc_mus-liver-zon-1.n5",
    "mito",
    chunks=(512, 512, 512),
)

In [13]:
import xarray as xr
import zarr

xr.open_zarr(
    "/nrs/cellmap/ackermand/cellmap/crop_jrc_mus-liver-zon-1.n5",
    "mito",
    chunks=(512, 512, 512),
)

/tmp/ipykernel_15576/1713413908.py:4: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  xr.open_zarr(


KeyError: 'Zarr object is missing the attribute `_ARRAY_DIMENSIONS` and the NCZarr metadata, which are required for xarray to determine variable dimensions.'

In [33]:
from fibsem_tools import read_xarray, read, read_dask
import xarray as xr

segmentation_xarray = read_xarray(
    "/nrs/cellmap/ackermand/cellmap/crop_jrc_mus-liver-zon-1.n5/mito",
    chunks=(512, 512, 512),
)


def get_skeleton(chunk):
    print(chunk)


xr.apply_ufunc(
    get_skeleton,
    segmentation_xarray,
    input_core_dims=[[dim], [dim]],
    dask="parallelized",
    output_dtypes=[float],
)

NameError: name 'dim' is not defined

In [43]:
chunk_sizes = segmentation_xarray.chunks

# Loop over chunks
for x_chunk in range(len(chunk_sizes[0])):
    for y_chunk in range(len(chunk_sizes[1])):
        for z_chunk in range(len(chunk_sizes[2])):
            # Select chunk using chunk indices
            chunk_to_load = segmentation_xarray.isel(x=x_chunk, y=y_chunk, z=z_chunk)

            # Load the selected chunk into memory
            a = chunk_to_load.load()
            break
        break
    break

In [3]:
from fibsem_tools import read_xarray, read, read_dask
import xarray as xr

segmentation_xarray = read_xarray(
    "/nrs/cellmap/ackermand/cellmap/crop_jrc_mus-liver-zon-1.n5/mito",
    chunks=(512, 512, 512),
)
segmentation_xarray

<xarray.DataArray 'mito' (z: 6000, y: 6000, x: 6000)>
dask.array<file:///nrs/cellmap/ackermand/cellmap/crop_jrc_mus-liver-zon-1.n5/mito, shape=(6000, 6000, 6000), dtype=uint64, chunksize=(512, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * z        (z) float64 0.0 8.0 16.0 24.0 ... 4.798e+04 4.798e+04 4.799e+04
  * y        (y) float64 0.0 8.0 16.0 24.0 ... 4.798e+04 4.798e+04 4.799e+04
  * x        (x) float64 0.0 8.0 16.0 24.0 ... 4.798e+04 4.798e+04 4.799e+04
Attributes:
    offset:           [61952, 35072, 120000]
    pixelResolution:  {'dimensions': [8.0, 8.0, 8.0], 'unit': 'nm'}

In [1]:
import dask.array as da

gt_da = da.from_zarr(
    "/nrs/cellmap/ackermand/cellmap/leaf-gall/jrc_22ak351-leaf-3m.n5",
    "larger_validation_crop",
    chunks=(256, 256, 256),
)

In [5]:
gt_da.blocks[0]

dask.array<blocks, shape=(256, 6464, 32690), dtype=uint16, chunksize=(256, 256, 256), chunktype=numpy.ndarray>

In [8]:
delayed_result = [
    dask.delayed(get_overlap_dict)(gt_block, test_block)
    for (gt_block, test_block) in zip(gt_da.blocks, test_da.blocks)
]

In [23]:
from funlib.geometry import Roi, Coordinate
from funlib.persistence import Array

r = Roi(
    (61952, 35072, 120000)[::-1],
    Coordinate(8 * 6000, 8 * 6000, 8 * 6000),
)

# create a struct for block_roi which contains a unique integer index as well as an Roi
from dataclasses import dataclass
from typing import Tuple
from funlib.geometry import Roi


@dataclass
class BlockInfo:
    index: int
    roi: Roi


def create_block_rois(roi: Roi, ds: Array, block_size=None):
    roi = roi.snap_to_grid(ds.chunk_shape * ds.voxel_size)
    if not block_size:
        block_size = ds.chunk_shape * ds.voxel_size

    block_rois = []
    index = 0
    for z in range(roi.get_begin()[2], roi.get_end()[2], block_size[2]):
        for y in range(roi.get_begin()[1], roi.get_end()[1], block_size[1]):
            for x in range(roi.get_begin()[0], roi.get_end()[0], block_size[0]):
                block_rois.append(BlockInfo(index, Roi((x, y, z), block_size)))
                index += 1

    return block_rois


from funlib.persistence import open_ds

ds = open_ds("/nrs/cellmap/ackermand/cellmap/crop_jrc_mus-liver-zon-1.n5", "mito")
ds.data_roi
block_rois = create_block_rois(r, ds)
block_rois[-1]

BlockInfo(index=110591, roi=[167936:168960, 82944:83968, 109568:110592] (1024, 1024, 1024))

In [1]:
from dask_skeletonization import Skeletonize
from funlib.persistence import open_ds
from funlib.geometry import Roi, Coordinate

ds = open_ds(
    "/nrs/cellmap/ackermand/cellmap/withFullPaths/jrc_mus-liver-zon-1/jrc_mus-liver-zon-1_postprocessed.zarr",
    "/mito/postprocessed_mito_fixed_filled_volumeFiltered",
)
sk = Skeletonize(
    ds,
    output_directory="/nrs/cellmap/ackermand/tmp/skeletons",
    total_roi=Roi(
        (61952, 35072, 120000)[::-1],
        Coordinate(8 * 600, 8 * 600, 8 * 600),
    ),
    read_write_roi=Roi((0, 0, 0), Coordinate(8 * 512, 8 * 512, 8 * 512)),
    num_workers=8,
)  # , Roi((61952+212*3*8, 35072+212*3*8, 120000+212*3*8)[::-1], Coordinate(8 * 212, 8 * 212, 8 * 212)) )
sk.get_skeletons()
# sk.process_chunked_skeletons("/nrs/cellmap/ackermand/tests/tmp/")

2024-04-16 15:58:59.464 INFO     Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-04-16 15:58:59.465 INFO     NumExpr defaulting to 8 threads.
2024-04-16 15:59:06.576 WARNING  primitive faces are immutable: not setting!


8


2024-04-16 15:59:09.227 INFO     To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2024-04-16 15:59:09.288 INFO     State start
2024-04-16 15:59:09.293 INFO       Scheduler at:     tcp://127.0.0.1:35467
2024-04-16 15:59:09.294 INFO       dashboard at:  http://127.0.0.1:8787/status
2024-04-16 15:59:09.295 INFO     Registering Worker plugin shuffle
2024-04-16 15:59:09.312 INFO             Start Nanny at: 'tcp://127.0.0.1:45629'
2024-04-16 15:59:13.452 INFO     Register worker <WorkerState 'tcp://127.0.0.1:44065', name: 0, status: init, memory: 0, processing: 0>
2024-04-16 15:59:13.460 INFO     Starting worker compute stream, tcp://127.0.0.1:44065
2024-04-16 15:59:13.462 INFO     Starting established connection to tcp://127.0.0.1:53388
2024-04-16 15:59:13.501 INFO     Receive client connection: Client-c8b84490-fc2b-11ee-ab21-04d4c424482a
2024-04-16 15:59:13.503 INFO     Starting established connection to tcp://127.0.

http://ackermand-ws2.hhmi.org:8787/status


2024-04-16 15:59:13.594 INFO             Start Nanny at: 'tcp://127.0.0.1:33235'
2024-04-16 15:59:13.602 INFO             Start Nanny at: 'tcp://127.0.0.1:43431'
2024-04-16 15:59:13.612 INFO             Start Nanny at: 'tcp://127.0.0.1:36259'
2024-04-16 15:59:13.621 INFO             Start Nanny at: 'tcp://127.0.0.1:33243'
2024-04-16 15:59:13.630 INFO             Start Nanny at: 'tcp://127.0.0.1:46675'
2024-04-16 15:59:13.641 INFO             Start Nanny at: 'tcp://127.0.0.1:39851'
2024-04-16 15:59:21.189 INFO     Register worker <WorkerState 'tcp://127.0.0.1:42867', name: 3, status: init, memory: 0, processing: 0>
2024-04-16 15:59:21.192 INFO     Starting worker compute stream, tcp://127.0.0.1:42867
2024-04-16 15:59:21.193 INFO     Starting established connection to tcp://127.0.0.1:39626
2024-04-16 15:59:21.205 INFO     Register worker <WorkerState 'tcp://127.0.0.1:45165', name: 2, status: init, memory: 0, processing: 0>
2024-04-16 15:59:21.206 INFO     Starting worker compute stream, 

http://ackermand-ws2.hhmi.org:8787/status


2024-04-16 16:00:14.302 INFO             Start Nanny at: 'tcp://127.0.0.1:35583'
2024-04-16 16:00:14.311 INFO             Start Nanny at: 'tcp://127.0.0.1:37029'
2024-04-16 16:00:14.317 INFO             Start Nanny at: 'tcp://127.0.0.1:41837'
2024-04-16 16:00:14.330 INFO             Start Nanny at: 'tcp://127.0.0.1:35913'
2024-04-16 16:00:14.338 INFO             Start Nanny at: 'tcp://127.0.0.1:45897'
2024-04-16 16:00:21.381 INFO     Register worker <WorkerState 'tcp://127.0.0.1:33209', name: 2, status: init, memory: 0, processing: 0>
2024-04-16 16:00:21.384 INFO     Starting worker compute stream, tcp://127.0.0.1:33209
2024-04-16 16:00:21.385 INFO     Starting established connection to tcp://127.0.0.1:56288
2024-04-16 16:00:21.392 INFO     Register worker <WorkerState 'tcp://127.0.0.1:33517', name: 5, status: init, memory: 0, processing: 0>
2024-04-16 16:00:21.394 INFO     Starting worker compute stream, tcp://127.0.0.1:33517
2024-04-16 16:00:21.395 INFO     Starting established conne

In [11]:
ds = open_ds(
    "/nrs/cellmap/ackermand/cellmap/withFullPaths/jrc_mus-liver-zon-1/jrc_mus-liver-zon-1_postprocessed.zarr",
    "/mito/postprocessed_mito_fixed_filled_volumeFiltered",
)
tmp = ds.to_ndarray(Roi((0, 0, 0), (16, 16, 16)))
print(tmp.dtype.byteorder, tmp)
tmp = tmp.newbyteorder().byteswap()
print(tmp.dtype.byteorder, tmp)
ds = open_ds("/nrs/cellmap/ackermand/cellmap/crop_jrc_mus-liver-zon-1.n5", "mito")
print(ds.to_ndarray(Roi((120000, 36000, 62000), (8, 8, 8))).dtype.byteorder)

> [[[0 0]
  [0 0]]

 [[0 0]
  [0 0]]]
< [[[0 0]
  [0 0]]

 [[0 0]
  [0 0]]]
=


In [1]:
from funlib.persistence import open_ds
from dask_skeletonization import Skeletonize
from funlib.geometry import Roi,Coordinate
ds = open_ds(
    "/nrs/cellmap/ackermand/cellmap/crop_jrc_mus-liver-zon-1.n5",
    "mito",
)
sk = Skeletonize(
    ds,
    output_directory="/nrs/cellmap/ackermand/tmp/skeletons",
    total_roi=Roi(
        (61952, 35072, 120000)[::-1],
        Coordinate(8 * 6000, 8 * 6000, 8 * 6000),
    ),
    read_write_roi=Roi((0, 0, 0), Coordinate(8 * 512, 8 * 512, 8 * 512)),
    num_workers=8,
)  # , Roi((61952+212*3*8, 35072+212*3*8, 120000+212*3*8)[::-1], Coordinate(8 * 212, 8 * 212, 8 * 212)) )
sk.get_skeletons()


2024-04-16 23:17:09.729 INFO     Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-04-16 23:17:09.731 INFO     NumExpr defaulting to 8 threads.
2024-04-16 23:17:17.742 WARNING  primitive faces are immutable: not setting!


1728


2024-04-16 23:17:20.545 INFO     To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2024-04-16 23:17:20.610 INFO     State start
2024-04-16 23:17:20.613 INFO     Found stale lock file and directory '/tmp/dask-scratch-space/scheduler-23at9g59', purging
2024-04-16 23:17:20.622 INFO       Scheduler at:     tcp://127.0.0.1:35303
2024-04-16 23:17:20.623 INFO       dashboard at:  http://127.0.0.1:8787/status
2024-04-16 23:17:20.625 INFO     Registering Worker plugin shuffle
2024-04-16 23:17:20.660 INFO             Start Nanny at: 'tcp://127.0.0.1:43201'
2024-04-16 23:17:20.672 INFO             Start Nanny at: 'tcp://127.0.0.1:45311'
2024-04-16 23:17:20.707 INFO             Start Nanny at: 'tcp://127.0.0.1:37391'
2024-04-16 23:17:20.716 INFO             Start Nanny at: 'tcp://127.0.0.1:46147'
2024-04-16 23:17:20.726 INFO             Start Nanny at: 'tcp://127.0.0.1:41497'
2024-04-16 23:17:20.737 INFO             Start Nan

http://ackermand-ws2.hhmi.org:8787/status


2024-04-16 23:17:41.015 INFO     Remove client Client-04a43163-fc69-11ee-9c6f-04d4c424482a
2024-04-16 23:17:41.016 INFO     Received 'close-stream' from tcp://127.0.0.1:55084; closing.
2024-04-16 23:17:41.020 INFO     Remove client Client-04a43163-fc69-11ee-9c6f-04d4c424482a
2024-04-16 23:17:41.034 INFO     Close client connection: Client-04a43163-fc69-11ee-9c6f-04d4c424482a


KeyboardInterrupt: 

2024-04-16 23:17:46.650 WARNING  primitive faces are immutable: not setting!
2024-04-16 23:17:46.680 WARNING  primitive faces are immutable: not setting!
2024-04-16 23:17:47.682 WARNING  primitive faces are immutable: not setting!
2024-04-16 23:17:47.687 WARNING  primitive faces are immutable: not setting!
2024-04-16 23:17:47.691 WARNING  primitive faces are immutable: not setting!
2024-04-16 23:17:47.703 WARNING  primitive faces are immutable: not setting!
2024-04-16 23:17:47.708 WARNING  primitive faces are immutable: not setting!
2024-04-16 23:17:47.712 WARNING  primitive faces are immutable: not setting!


In [9]:
read_write_roi = Roi((0, 0, 0), Coordinate(8 * 512, 8 * 512, 8 * 512))
read_write_roi.shape

# swap byte order if little endian

(4096, 4096, 4096)